In [1]:
import requests
import json
import pandas as pd
from tqdm import tqdm
import unicodedata


In [29]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
}
params = {
    "limit": "48", # limit number of items each page
    "include": "advertisement",
    "aggregation": "2",
    "category": "1846",
    #"page": "1", # page number
    "urlkey": "laptop-may-vi-tinh-linh-kien" # product category
}
new_api_request_url = "https://tiki.vn/api/v2/products" # new api from tiki developer
old_api_request_url = "https://tiki.vn/api/personalish/v1/blocks/listings" # current api from tiki webpage
#pageNum = 1
product_id_list = []
df_id = pd.DataFrame([], columns=["id"])
# old api page: 2000 - 209   2000 - 209
# old api no page: 47376 - 988
# new-api-page : 2000 - 198  2000 - 199  2000 - 240 
# new-api-no page:  10080 - 211    8400 - 176

<b>Fetch Product ID Data<b>

In [30]:
while True:       
    '''With the page number param available, data return only 2000 items -> I will block it to get as much data as possible''' 
    #print("Crawl page number: ", pageNum)
    #params["page"] = pageNum
    response = requests.get(new_api_request_url, headers=headers, params=params)
    if response.status_code == 200:
        #print("Crawling data successfully")
        #products = json.loads(response.text)["data"]
        for product in response.json()["data"]:
            id = str(product["id"])
            product_id_list.append(id)
    else:
        print(f"Error, status code = {response.status_code}")
        break

    # Divided each batch with maximum 2000 items 
    if(len(product_id_list) % 2000 == 0):
        print("Reach 2000 items")
        df_temp = pd.DataFrame(product_id_list, columns=["id"])
        df_id = pd.concat([df_id, df_temp], ignore_index=True)
        # Truncate all the list
        del product_id_list[:]
        print(len(product_id_list))
        
    #pageNum+=1

Reach 2000 items
0


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [31]:
print(len(product_id_list))
df_id.shape

4320


(6000, 1)

In [32]:
# Fill out the list that not reach 2000 items
df_temp = pd.DataFrame(product_id_list, columns=["id"])
df_id = pd.concat([df_id, df_temp], ignore_index=True)
# Truncate all the list
del product_id_list[:]
print(len(product_id_list))
df_id.shape

0


(10320, 1)

In [33]:
df_id.head()

,id
0,38323087
1,26314426
2,134561311
3,8608925
4,46634155


In [34]:
list_temp = df_id["id"].values.tolist()

In [35]:
print(len(list_temp))

10320


<b> Fetch Product Details Data <b>

In [75]:
url = "https://tiki.vn/api/v2/products/{}"
pars = {"platform": "web", 
        "spid": "124939771"
}
product_detail_list = []
product_json_list = []
df_product = pd.DataFrame([])

In [76]:
for id in tqdm(list_temp, total=len(list_temp)):
    response = requests.get('https://tiki.vn/api/v2/products/{}'.format(id), headers=headers, params=pars)
    if(response.status_code==200):
        #----------------
        try:
            #product_detail_list.append(response.text)
            product = json.loads(response.text)         
            product_detail_list.append(product)
        except:
            #print("Something else went wrong")
            pass

    else:
        print(f"Error, status code = {response.status_code}")
        pass

    # Divided each batch with maximum 2000 items 
    if(len(product_detail_list) == 2000):
        print("Reach 2000 items")
        df_temp = pd.DataFrame.from_dict(product_detail_list)
        df_product = pd.concat([df_product, df_temp], ignore_index=True)
        # Truncate all the list
        del product_detail_list[:]
        del product_json_list[:]
        print(len(product_detail_list), len(product_json_list))
    
        

 30%|███       | 3120/10320 [16:14<36:55,  3.25it/s]  

Reach 2000 items
0 0


 62%|██████▏   | 6446/10320 [33:05<22:42,  2.84it/s]  

Reach 2000 items
0 0


 94%|█████████▍| 9738/10320 [49:49<03:35,  2.70it/s]

Reach 2000 items
0 0


100%|██████████| 10320/10320 [52:52<00:00,  3.25it/s]


In [78]:
print(len(product_detail_list))
df_product.shape

367


(6000, 70)

In [80]:
# Fill out the list that not reach 2000 items
df_temp = pd.DataFrame.from_dict(product_detail_list)
df_product = pd.concat([df_product, df_temp], ignore_index=True)
# Truncate all the list
del product_detail_list[:]
print(len(product_detail_list))
df_product.shape

0


(6367, 70)

In [81]:
df_product.head()

,id,master_id,sku,name,url_key,url_path,short_url,type,book_cover,short_description,...,inventory,warranty_info,return_and_exchange_policy,is_tier_pricing_available,is_tier_pricing_eligible,asa_cashback_widget,benefits,price_comparison,video_url,best_price_guaranteed
0,38323087,38323087,9386902161867,(Giá Hủy Diệt) Ổ Cứng Di Động HDD WD My Passpo...,o-cung-di-dong-wd-my-passport-model-2019-hang-...,o-cung-di-dong-wd-my-passport-model-2019-hang-...,https://tiki.vn/product-p38323087.html?spid=53...,configurable,None,Thông số kỹ thuật WD My Passport màu đen mode...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Thời gian bảo hành', 'value': '36 T...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 213, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,NaN,NaN,NaN
1,26314426,26314426,9365508570174,Dây Cáp Mạng Cat6 Bấm Sẵn 2 đầu,day-cap-mang-cat6-bam-san-2-dau-p26314426,day-cap-mang-cat6-bam-san-2-dau-p26314426.html...,https://tiki.vn/product-p26314426.html?spid=14...,configurable,None,+ Cáp mạng CAT6 UTP được sử dụng chất liệu CCA...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Hướng dẫn bảo hành', 'value': 'Xem ...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 213, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,"{'sub_title': 'Giá từ 32.650 ₫', 'title': '6 n...",NaN,NaN
2,134561311,134561311,8367091212771,USB 3.0 SanDisk Ultra Flair CZ73 - Hàng Nhập Khẩu,usb-3-0-sandisk-ultra-flair-cz73-64gb-hang-nha...,usb-3-0-sandisk-ultra-flair-cz73-64gb-hang-nha...,https://tiki.vn/product-p134561311.html?spid=9...,configurable,None,Phần mềm bảo vệ và chống truy cập trái phép Se...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Thời gian bảo hành', 'value': '5 Nă...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 213, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,NaN,NaN,NaN
3,8608925,8608925,6460064428402,Hộp mực dành cho máy in Laser Brother HL-L2321D.,hop-muc-may-in-laser-brother-hl-l2321d-p8608925,hop-muc-may-in-laser-brother-hl-l2321d-p860892...,https://tiki.vn/product-p8608925.html?spid=263...,simple,None,Hộp mực TN-2385/2325... hàng thay thế thích hợ...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Thời gian bảo hành', 'value': '1 Th...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 213, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,"{'sub_title': 'Giá từ 107.900 ₫', 'title': '9 ...",NaN,NaN
4,46634155,46634155,7172023663098,Dây Cáp Mạng Cat6 Bấm Sẵn 2 đầu,day-cap-mang-cat6-bam-san-2-dau-p46634155,day-cap-mang-cat6-bam-san-2-dau-p46634155.html...,https://tiki.vn/product-p46634155.html?spid=16...,configurable,None,Vỏ bọc bên ngoài làm bằng nhựa PVC nguyên chất...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Hướng dẫn bảo hành', 'value': 'Xem ...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 213, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,"{'sub_title': 'Giá từ 24.900 ₫', 'title': '2 n...",NaN,NaN


In [ ]:
# print(df_product.shape) 
#df_product.to_csv("product_full_details.csv", encoding='utf-8-sig', index=False)

In [24]:
df = pd.read_csv('product_full_details.csv')
df.head()

,id,master_id,sku,name,url_key,url_path,short_url,type,book_cover,short_description,...,inventory,warranty_info,return_and_exchange_policy,is_tier_pricing_available,is_tier_pricing_eligible,asa_cashback_widget,benefits,price_comparison,video_url,best_price_guaranteed
0,38323087,38323087,9.386900e+12,(Giá Hủy Diệt) Ổ Cứng Di Động HDD WD My Passpo...,o-cung-di-dong-wd-my-passport-model-2019-hang-...,o-cung-di-dong-wd-my-passport-model-2019-hang-...,https://tiki.vn/product-p38323087.html?spid=53...,configurable,NaN,Thông số kỹ thuật WD My Passport màu đen mode...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Thời gian bảo hành', 'value': '36 T...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 213, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,NaN,NaN,NaN
1,26314426,26314426,9.365510e+12,Dây Cáp Mạng Cat6 Bấm Sẵn 2 đầu,day-cap-mang-cat6-bam-san-2-dau-p26314426,day-cap-mang-cat6-bam-san-2-dau-p26314426.html...,https://tiki.vn/product-p26314426.html?spid=14...,configurable,NaN,+ Cáp mạng CAT6 UTP được sử dụng chất liệu CCA...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Hướng dẫn bảo hành', 'value': 'Xem ...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 213, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,"{'sub_title': 'Giá từ 32.650 ₫', 'title': '6 n...",NaN,NaN
2,134561311,134561311,8.367090e+12,USB 3.0 SanDisk Ultra Flair CZ73 - Hàng Nhập Khẩu,usb-3-0-sandisk-ultra-flair-cz73-64gb-hang-nha...,usb-3-0-sandisk-ultra-flair-cz73-64gb-hang-nha...,https://tiki.vn/product-p134561311.html?spid=9...,configurable,NaN,Phần mềm bảo vệ và chống truy cập trái phép Se...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Thời gian bảo hành', 'value': '5 Nă...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 213, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,NaN,NaN,NaN
3,8608925,8608925,6.460060e+12,Hộp mực dành cho máy in Laser Brother HL-L2321D.,hop-muc-may-in-laser-brother-hl-l2321d-p8608925,hop-muc-may-in-laser-brother-hl-l2321d-p860892...,https://tiki.vn/product-p8608925.html?spid=263...,simple,NaN,Hộp mực TN-2385/2325... hàng thay thế thích hợ...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Thời gian bảo hành', 'value': '1 Th...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 213, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,"{'sub_title': 'Giá từ 107.900 ₫', 'title': '9 ...",NaN,NaN
4,46634155,46634155,7.172020e+12,Dây Cáp Mạng Cat6 Bấm Sẵn 2 đầu,day-cap-mang-cat6-bam-san-2-dau-p46634155,day-cap-mang-cat6-bam-san-2-dau-p46634155.html...,https://tiki.vn/product-p46634155.html?spid=16...,configurable,NaN,Vỏ bọc bên ngoài làm bằng nhựa PVC nguyên chất...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Hướng dẫn bảo hành', 'value': 'Xem ...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 213, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,"{'sub_title': 'Giá từ 24.900 ₫', 'title': '2 n...",NaN,NaN


In [25]:
df_tmp = df[['current_seller', 'all_time_quantity_sold', 'id', 'brand', 'categories']]
df_tmp.rename(columns={'id': 'product_id'}, inplace=True)
df_tmp.shape # 6367 

C:\Users\nthph\AppData\Local\Temp\ipykernel_20768\292559163.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp.rename(columns={'id': 'product_id'}, inplace=True)


(6367, 5)

In [26]:
# Create dim seller
seller = df_tmp['current_seller'].tolist()
seller_list = [eval(p) for p in seller]
df_seller = pd.DataFrame.from_dict(seller_list)
df_seller = df_seller[['id', 'sku', 'name', 'link']]
df_seller.rename(columns={'id': 'seller_id'}, inplace=True)
# fact sale - seller id - product id
df_fact = pd.concat([df_tmp, df_seller], axis=1, ignore_index=False)
df_fact.drop(['sku', 'name', 'link', 'current_seller', 'brand', 'categories'], axis=1, inplace=True)


In [28]:
# Clean dim seller
print(df_seller.shape) # 6367 
df_seller_dim = df_seller.drop_duplicates(keep='first')
print(df_seller_dim.shape) # 48
df_seller_dim.head()

(6367, 4)
(48, 4)


,seller_id,sku,name,link
0,25203,9490939673610,Lagihitech2,https://tiki.vn/cua-hang/lagihitech2
1,73012,5712899283503,SiroShop,https://tiki.vn/cua-hang/siroshop
2,783,6109891952284,Lưu Trữ Sài Gòn,https://tiki.vn/cua-hang/luu-tru-sai-gon
3,65686,2676868629639,Mực in máy in Nhật Hưng,https://tiki.vn/cua-hang/muc-in-may-in-vinh-hung
4,73012,3867466908904,SiroShop,https://tiki.vn/cua-hang/siroshop


In [29]:
# Create dim category
category = df_tmp['categories'].tolist()
category_list = [eval(p) for p in category]
# Tạo url cho subcategory của sp
for item in category_list:
    id = item['id']
    name = item['name']
    # Xử lý chuỗi Tiếng Việt
    name = name.lower()
    name = name.replace("đ", "d").replace(" ", "-")
    name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore')
    name = name.decode('utf-8')
    item['url'] = f"/{name}/c{id}"

df_category = pd.DataFrame.from_dict(category_list)
df_category = df_category[['id','name', 'url']]
df_category.rename(columns={'id': 'category_id'}, inplace=True)
# fact sale - seller_id - product_id - category_id
df_fact = pd.concat([df_fact, df_category], axis=1, ignore_index=False)
df_fact.drop(['name', 'url'], axis=1, inplace=True)


In [30]:
# Clean dim category
print(df_category.shape) # 6367 
df_category_dim = df_category.drop_duplicates(keep='first')
print(df_category_dim.shape) # 17
df_category_dim.head()

(6367, 3)
(17, 3)


,category_id,name,url
0,28842,Ổ Cứng HDD Di Động,/o-cung-hdd-di-dong/c28842
1,4642,Phụ Kiện Thiết Bị Mạng,/phu-kien-thiet-bi-mang/c4642
2,1828,USB,/usb/c1828
3,28944,Drums và Toner Cho Máy In Laser,/drums-va-toner-cho-may-in-laser/c28944
6,5350,Case Ổ Cứng- Hộp Đựng HDD Box - Dock Ổ Cứng,/case-o-cung--hop-dung-hdd-box---dock-o-cung/c...


In [31]:
# Create dim brand
brand = df_tmp['brand'].tolist()
brand_list = [eval(p) for p in brand]
df_brand = pd.DataFrame.from_dict(brand_list)
df_brand = df_brand[['id', 'name']]
df_brand.rename(columns={'id': 'brand_id'}, inplace=True)
# fact sale - seller_id - product_id - category_id - brand_id
df_fact = pd.concat([df_fact, df_brand], axis=1, ignore_index=False)
df_fact.drop(['name'], axis=1, inplace=True)


In [32]:
# Clean dim brand
print(df_brand.shape) # 6367
df_brand_dim = df_brand.drop_duplicates(keep='first')
print(df_brand_dim.shape) # 17
df_brand_dim.head()

(6367, 2)
(17, 2)


,brand_id,name
0,21862,WD
1,111461,OEM
2,18845,SanDisk
5,195495,Kingmaster
6,110682,Orico


In [33]:
# Create dim product 
df_product = df[['id', 'name', 'sku', 'price', 'list_price', 'discount', 'discount_rate', 'inventory_status', 'stock_item']]
df_product.head()

,id,name,sku,price,list_price,discount,discount_rate,inventory_status,stock_item
0,38323087,(Giá Hủy Diệt) Ổ Cứng Di Động HDD WD My Passpo...,9.386900e+12,1385000,1390000,5000,0,available,"{'max_sale_qty': 1000, 'min_sale_qty': 1, 'pre..."
1,26314426,Dây Cáp Mạng Cat6 Bấm Sẵn 2 đầu,9.365510e+12,75000,75000,0,0,available,"{'max_sale_qty': 1000, 'min_sale_qty': 1, 'pre..."
2,134561311,USB 3.0 SanDisk Ultra Flair CZ73 - Hàng Nhập Khẩu,8.367090e+12,110000,110000,0,0,available,"{'max_sale_qty': 1000, 'min_sale_qty': 1, 'pre..."
3,8608925,Hộp mực dành cho máy in Laser Brother HL-L2321D.,6.460060e+12,111900,111900,0,0,available,"{'max_sale_qty': 1000, 'min_sale_qty': 1, 'pre..."
4,46634155,Dây Cáp Mạng Cat6 Bấm Sẵn 2 đầu,7.172020e+12,48000,48000,0,0,available,"{'max_sale_qty': 1000, 'min_sale_qty': 1, 'pre..."


In [34]:
# stock_item: max_sale_qty, qty
stock = df_product['stock_item'].tolist()
stock_list = [eval(p) for p in stock]
df_stock = pd.DataFrame.from_dict(stock_list)
df_stock = df_stock[['max_sale_qty', 'qty']]
# Dim Product
df_product = pd.concat([df_product, df_stock], axis=1, ignore_index=False)
df_product.drop(['stock_item'], axis=1, inplace=True)

In [35]:
# Clean Dim Product
print(df_product.shape) # 6367 
df_product_dim = df_product.drop_duplicates(keep='first')
print(df_product_dim.shape) # 48 
df_product_dim.head()

(6367, 10)
(48, 10)


,id,name,sku,price,list_price,discount,discount_rate,inventory_status,max_sale_qty,qty
0,38323087,(Giá Hủy Diệt) Ổ Cứng Di Động HDD WD My Passpo...,9.386900e+12,1385000,1390000,5000,0,available,1000,1000
1,26314426,Dây Cáp Mạng Cat6 Bấm Sẵn 2 đầu,9.365510e+12,75000,75000,0,0,available,1000,1000
2,134561311,USB 3.0 SanDisk Ultra Flair CZ73 - Hàng Nhập Khẩu,8.367090e+12,110000,110000,0,0,available,1000,1000
3,8608925,Hộp mực dành cho máy in Laser Brother HL-L2321D.,6.460060e+12,111900,111900,0,0,available,1000,1000
4,46634155,Dây Cáp Mạng Cat6 Bấm Sẵn 2 đầu,7.172020e+12,48000,48000,0,0,available,1000,1000


In [36]:
# We dont drop duplicates fact table since it involves to other attributes that we dont extract them to the schema
print(df_fact.shape) # 6367 
df_fact.head()

(6367, 5)


,all_time_quantity_sold,product_id,seller_id,category_id,brand_id
0,2244,38323087,25203,28842,21862
1,1120,26314426,73012,4642,111461
2,9979,134561311,783,1828,18845
3,2638,8608925,65686,28944,111461
4,490,46634155,73012,4642,111461


In [37]:
df_fact.to_csv('dataset/fact_sale.csv')
df_product_dim.to_csv('dataset/dim_product.csv')
df_brand_dim.to_csv('dataset/dim_brand.csv')
df_seller_dim.to_csv('dataset/dim_seller.csv')
df_category_dim.to_csv('dataset/dim_category.csv')